In [ ]:
import pyterrier as pt
if not pt.started():
  pt.init()

     |████████████████████████████████| 126 kB 7.7 MB/s 
     |████████████████████████████████| 95 kB 3.8 MB/s 
     |████████████████████████████████| 1.1 MB 52.2 MB/s 
     |████████████████████████████████| 69 kB 8.8 MB/s 
     |████████████████████████████████| 222 kB 56.4 MB/s 
     |████████████████████████████████| 6.3 MB 57.3 MB/s 
     |████████████████████████████████| 596 kB 58.9 MB/s 
     |████████████████████████████████| 294 kB 69.4 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 1.8 MB 44.4 MB/s 
     |████████████████████████████████| 291 kB 69.7 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.7.1-py3-none-any.whl size=102452 sha256=e4a1648fdc598f59c7caafab912304bc93587b179424d49fb4321d3f024c1e9e
  Stored in directory: /root/.cache/pip/wheels/c0/9a/c8/1c2d9ec6a1494bb54f47e0d2627b5ed7b2de704b66723d3417
  Created wheel for ir-measures: filena

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/workspace/corpus.csv')

In [ ]:
df

,docno,text
0,S1f6b58eb-A5c530110,"This week I have my hair in braids, much like ..."
1,S1a9db4fc-Acc4206f5,"On November 10, UN Women elected its first exe..."
2,S1a924cd4-A994e77ca,"Madam, October is Women's History Month and al..."
3,S1a3be6ac-A873cd7aa,"I rise today to recognize Frederick Couchie, f..."
4,S1beff81f-A1206f3ee,I find the continued existence of and the incr...
...,...,...
338615,Sca72da7d-A5cd811ac,"Thanks Lifeisgood, a pleasure to debate you to..."
338616,Sca72da7d-Af008e63e,"It is a a great pleasure to debate you, feveri..."
338617,Sca72da7d-Ac77ea7a1,"Oh, a forfeit. How disappointing, I didn't see..."
338618,Sca72da7d-Ae83ab999,Thanks to my opponent for instigating this deb...


In [ ]:
from xml.etree import ElementTree
tree = ElementTree.parse('/content/drive/MyDrive/workspace/topics.xml')
data = tree.getroot()
titles = []
title_ids = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    title_ids.append(num)
topics = pd.DataFrame(titles,columns = ['query'])
topics['qid'] = topics.index + 51
columns_titles = ["qid","query"]
topics=topics.reindex(columns=columns_titles)

In [ ]:
qrels_path=("/content/drive/MyDrive/workspace/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

In [ ]:
docnos = qrels.docno
# list(docnos)

In [ ]:
df_test = df.copy()
df_test = df_test[df_test['docno'].isin(docnos)]
df_test

,docno,text
143,Sa4c647b7-A684d37f3,Bullfighting is too dangerous to humans to jus...
144,Sa4c647b7-Abf0df4fc,Bullfighting constitutes animal torture becaus...
145,Sa4c647b7-A664695d4,"Many matadors are gored each year. In 2010, fa..."
146,Sa4c647b7-A75929e8c,"Robert Elms argued in 2010 that ""Those who see..."
148,Sa4c647b7-A91d0525,Harming animals for entertainment is immoralal...
...,...,...
337759,Safa455cc-A3bd6a0ca,Nuclear energy is a reasonable approach to gen...
338026,S51231cdf-Aaf2014c7,Media in general does not need to be rated. Th...
338223,Se34330e5-Aa7bf18da,My position is that concealed weapon laws and ...
338410,Scdea971a-A6a6d8e98,Communism is not the only government that soun...


In [ ]:
pd_indexer = pt.DFIndexer("./pd_index")
indexref = pd_indexer.index(df_test["text"], df_test["docno"])

In [ ]:
index = pt.IndexFactory.of(indexref)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)

In [ ]:
res

,qid,docid,docno,rank,score,query
0,51,2523,S885c6b4f-Ad3259e81,0,12.112106,do we need sex education in schools
1,51,1963,S428a8bea-Aba4c7b80,1,11.959569,do we need sex education in schools
2,51,1773,S471f6d0a-A6a764709,2,11.783293,do we need sex education in schools
3,51,1866,S11b834b9-Aabe96a35,3,11.758555,do we need sex education in schools
4,51,2691,S937b1a96-A198bbc54,4,11.745463,do we need sex education in schools
...,...,...,...,...,...,...
33622,100,2366,Sbe07bde7-Ae09eaaa5,995,0.868738,do we need cash
33623,100,933,Sf81067da-A42e35728,996,0.867737,do we need cash
33624,100,468,S86d37902-Acbd25d3c,997,0.865740,do we need cash
33625,100,1541,Sf22d60dd-A51890018,998,0.865740,do we need cash


In [ ]:
eval = pt.Utils.evaluate(res,qrels,metrics=["ndcg_cut_5", "bpref"])
eval

{'bpref': 0.6375908840552834, 'ndcg_cut_5': 0.6683541337521044}

In [ ]:
pd_indexer = pt.DFIndexer("./pd_index_full")
indexref = pd_indexer.index(df["text"], df["docno"])

16:16:05.790 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (Sdbc891ec-A55b98352) - further warnings are suppressed
16:20:28.497 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 57 empty documents
16:20:29.053 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key Saeff5ad8-Afba459ff is not unique: 160773,160770
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:369)


In [ ]:
index = pt.IndexFactory.of(indexref)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)

In [ ]:
eval = pt.Utils.evaluate(res,qrels,metrics=["ndcg_cut_5", "bpref"])
eval

{'bpref': 0.6261477227722415, 'ndcg_cut_5': 0.5527540851759042}